<a href="https://colab.research.google.com/github/PrithvirajChauhan1/Stockmarket/blob/main/Stock_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install keras


In [ ]:
import math
import pandas_datareader as web
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
plt.style.use("fivethirtyeight")


In [ ]:

df=web.DataReader('HDFCBANK.NS',data_source='yahoo',start='2012-01-01',end='2021-08-11')

RemoteDataError: ignored

In [ ]:
df.shape

NameError: ignored

In [ ]:
plt.figure(figsize=(16,8))
plt.title("Volume")
plt.plot(df['Volume'])
plt.xlabel('Time',fontsize=18)
plt.ylabel('Volume',fontsize=18)
plt.show()


In [ ]:
plt.figure(figsize=(31,8))
plt.title("Closing Price")
plt.plot(df['Close'])
plt.xlabel('Time',fontsize=18)
plt.ylabel('Close',fontsize=18)
plt.show()

In [ ]:
plt.figure(figsize=(16,8))
plt.title("High")
plt.plot(df['High'])
plt.xlabel('High',fontsize=18)
plt.ylabel('Volume',fontsize=18)
plt.show()

In [ ]:
data=df.filter(['Close'])
dataset=data.values
training_data_len=math.ceil(len(dataset)*0.8)
training_data_len

In [ ]:
scaler=MinMaxScaler(feature_range=(0,1))
scaled_data=scaler.fit_transform(dataset)
scaled_data

In [ ]:
train_data=scaled_data[0:training_data_len,:]
x_train=[]
y_train=[]
for i in range(60,len(train_data)):
  x_train.append(train_data[i-60:i,0])
  y_train.append(train_data[i,0])

  if i<=60:
    print(x_train)
    print(y_train)
    print()


In [ ]:
x_train,y_train=np.array(x_train),np.array(y_train)


In [ ]:
x_train=np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))
x_train.shape

In [ ]:
model=Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(x_train.shape[1],1)))
model.add(LSTM(50,return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))



In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',loss='mean_squared_error')

In [ ]:
model.fit(x_train,y_train,batch_size=1,epochs=10)

In [ ]:
test_data=scaled_data[training_data_len-60:,:]

x_test=[]
y_test=dataset[training_data_len:,:]
for i in range(60,len(test_data)):
    x_test.append(test_data[i-60:i,0])

In [ ]:
x_test=np.array(x_test)
x_test.shape

In [ ]:
x_test=np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))

In [ ]:
predictions=model.predict(x_test)
predictions= scaler.inverse_transform(predictions)

In [ ]:
#Error Root-mean Squared
rmse=np.sqrt(np.mean(predictions-y_test)**2)
rmse*100

In [ ]:
train=data[:training_data_len]
valid=data[training_data_len:]
valid['Predictions']=predictions
#Graphs
plt.figure(figsize=(31,8))
plt.title("Closing price")
plt.xlabel('Time')
plt.ylabel('Closing price Rupees')
plt.plot(train['Close'])
plt.plot(valid[['Close','Predictions']])
plt.legend(['Train','Val','Predictions'],loc='lower right')
plt.show

In [ ]:
valid